In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

YOLOV8

In [2]:
try:
    from ultralytics import YOLO
    print('ok')
except ModuleNotFoundError:
    !pip install ultralytics
    from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found exi

In [3]:
model=YOLO("yolov8l.pt")
print("Model is ready hehe")

100%|██████████| 83.7M/83.7M [00:00<00:00, 258MB/s]


Model is ready hehe


Tien xu li

In [4]:
import os

def remove_duplicates_ignore_label(label_folder):  
    prefix_img = ["IMG_2265", "IMG_2266", "IMG_2267", "IMG_2268", "IMG_2269",
                  "IMG_2270", "IMG_2271", "IMG_2272", "IMG_2273", "IMG_2274","IMG_1590"]
    
    for label_file in os.listdir(label_folder):
        if not label_file.endswith(".txt"):
            continue
        if any(label_file.startswith(prefix) for prefix in prefix_img):
            file_path = os.path.join(label_folder, label_file)
            seen_lines = set()
            
            with open(file_path, "r") as f:
                lines = f.readlines()
            
            unique_lines = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue

                label = parts[0]
                coords = tuple(map(float, parts[1:]))

                if coords not in seen_lines:
                    unique_lines.append(f"{label} {coords[0]} {coords[1]} {coords[2]} {coords[3]}\n")
                    seen_lines.add(coords)

            with open(file_path, "w") as f:
                f.writelines(unique_lines)

    print("✅ Removed duplicates for specified images.")

def insert_line_510_img_1581(label_folder):
    for label_file in os.listdir(label_folder):
        if label_file.startswith("IMG_1581") and label_file.endswith(".txt"):
            file_path = os.path.join(label_folder, label_file)
            with open(file_path, "r") as f:
                lines = f.readlines()

            insert_line = "1 0.719956 0.880197 0.018182 0.012377\n"
            index_to_insert = min(510, len(lines))  # để tránh lỗi nếu file ngắn hơn 510 dòng
            lines.insert(index_to_insert, insert_line)

            with open(file_path, "w") as f:
                f.writelines(lines)

    print("✅ Inserted line into IMG_1581 file.")


In [5]:
import shutil

input_labels = "/kaggle/input/training-set/training-set/Trainning_SET/labels"
working_labels = "/kaggle/working/labels"

if not os.path.exists(working_labels):
    shutil.copytree(input_labels, working_labels)


In [6]:
label_folder = "/kaggle/working/labels"

remove_duplicates_ignore_label(label_folder)
insert_line_510_img_1581(label_folder)

✅ Removed duplicates for specified images.
✅ Inserted line into IMG_1581 file.


In [7]:
import os

label_dir = "/kaggle/working/labels"
total_files = 0
total_boxes_class_0 = 0

for root, _, files in os.walk(label_dir):
    for file in files:
        if file.endswith(".txt"):
            file_path = os.path.join(root, file)
            total_files += 1
            with open(file_path, "r") as f:
                lines = f.readlines()
            class_0_lines = [line for line in lines if line.strip().startswith("0 ")]
            total_boxes_class_0 += len(class_0_lines)
            with open(file_path, "w") as f:
                f.writelines(class_0_lines)

print(f"✅ Đã xử lý {total_files} file label.")
print(f"📦 Tổng số box class 0 còn lại: {total_boxes_class_0}")


✅ Đã xử lý 5200 file label.
📦 Tổng số box class 0 còn lại: 556762


Chia du lieu

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

image_dir = '/kaggle/input/training-set/training-set/Trainning_SET/images'
label_dir = '/kaggle/working/labels'

output_dir = '/kaggle/working'
train_image_dir = os.path.join(output_dir, 'train/images')
val_image_dir = os.path.join(output_dir, 'val/images')
test_image_dir = os.path.join(output_dir, 'test/images')

train_label_dir = os.path.join(output_dir, 'train/labels')
val_label_dir = os.path.join(output_dir, 'val/labels')
test_label_dir = os.path.join(output_dir, 'test/labels')

for dir_path in [train_image_dir, val_image_dir, test_image_dir,
                 train_label_dir, val_label_dir, test_label_dir]:
    os.makedirs(dir_path, exist_ok=True)

image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]
label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

assert len(image_files) == len(label_files), "Số lượng ảnh và nhãn không khớp!"

train_images, test_val_images = train_test_split(image_files, test_size=0.3, random_state=42)
val_images, test_images = train_test_split(test_val_images, test_size=0.5, random_state=42)

def move_files(files, src_image_dir, src_label_dir, dest_image_dir, dest_label_dir):
    for file in files:
        image_path = os.path.join(src_image_dir, file)
        label_path = os.path.join(src_label_dir, file.replace('.jpg', '.txt').replace('.png', '.txt'))
        
        if os.path.exists(image_path) and os.path.exists(label_path):
            shutil.copy(image_path, os.path.join(dest_image_dir, file))
            shutil.copy(label_path, os.path.join(dest_label_dir, file.replace('.jpg', '.txt').replace('.png', '.txt')))

move_files(train_images, image_dir, label_dir, train_image_dir, train_label_dir)
move_files(val_images, image_dir, label_dir, val_image_dir, val_label_dir)
move_files(test_images, image_dir, label_dir, test_image_dir, test_label_dir)

print("Dataset đã được chia xong:")
print(f"- Train: {len(train_images)} ảnh")
print(f"- Val: {len(val_images)} ảnh")
print(f"- Test: {len(test_images)} ảnh")


Tạo file YAML

In [ ]:
try:
    import yaml
except ModuleNotFoundError:
    !pip install pyyaml
    import yaml

In [ ]:
config = {
    'path':  '/kaggle/working/',
    'train': 'train',
    'val':   'val',
    'test':  'test',
    'nc':    2,                     # 2 lớp
    'names': ['marked', 'unmarked'] # 0 → marked, 1 → unmarked
}

yaml_path = '/kaggle/working/data.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(config, f)

print('Done')

In [ ]:
# train_results = model.train(
#     data=yaml_path,  # Path to dataset configuration file
#     epochs=10,  # Number of training epochs
#     imgsz=640,  # Image size for training
#     batch=8,  # Device to run on (e.g., 'cpu', 0, [0,1,2,3])
#     device=0,
#     resume=True
# )

In [ ]:
train_results = model.train(
    data=yaml_path,  # Path to dataset configuration file
    epochs=20,  # Number of training epochs
    imgsz=800,  # Image size for training
    batch=4,  # Device to run on (e.g., 'cpu', 0, [0,1,2,3])
    device=0,
    conf=0.05,
    iou=0.3,
    resume=False
)

In [ ]:
metrics = model.val(conf=0.05,iou=0.3)


In [ ]:
model=YOLO('/kaggle/working/runs/detect/train/weights/best.pt')

In [ ]:
results = model.predict(source="/kaggle/working/test/images", conf=0.3, save=False)


In [ ]:
import cv2
def predict_and_save_images(output_folder,results,conf=0.5):

    os.makedirs(output_folder, exist_ok=True)


    for i, result in enumerate(results):
        image = result.orig_img.copy()
        boxes = result.boxes.xyxy.cpu().numpy()  # bbox: [x1, y1, x2, y2]

        for box in boxes:
            x1, y1, x2, y2 = map(int, box[:4])
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # bbox màu xanh

        # Tên ảnh gốc
        image_name = os.path.basename(result.path)
        output_path = os.path.join(output_folder, image_name)

        cv2.imwrite(output_path, image)
        print(f"Đã lưu: {output_path}")

In [ ]:
ouput_folder='/kaggle/working/predict'
predict_and_save_images(ouput_folder,results)
